# Using a Double Exponential Moving Average (DEMA) strategy to algorithmically trade stocks through the Alpaca API  
Credits to:  
-https://alpaca.markets/learn/pythonanywhere-trading-algorithm/  
-https://www.youtube.com/watch?v=bdlwwmVWfPg&list=PLBhJnyA0V0uIP6tScPs01FW5WtSpJdmcv&index=65&t=1148s

In [ ]:
!pip install alpaca-trade-api

In [2]:
import alpaca_trade_api as tradeapi

endpoint = 'https://paper-api.alpaca.markets'
key = '' # put your key here
secret = '' # put your secret key here

api = tradeapi.REST(base_url=endpoint, key_id=key, secret_key=secret)

account = api.get_account()

Setting environment variables locally: https://forum.alpaca.markets/t/how-to-set-enviroment-variables/1989/6  
Note: If you set it through the command prompt, I read somewhere that you should open a new command prompt afterwards.

In [11]:
import time
import datetime
from datetime import timedelta
from pytz import timezone

#Global variables
EST = timezone('EST')
symbols = ['VOX', 'VCR', 'IYK', 'FENY', 'FNCL', 'FHLC', 'FIDU', 'PAVE', 'FMAT', 'BBRE', 'XSW', 'FUTY']
timeframe = '5Min'
fast = 20
slow = 50

import logging
logging.basicConfig(filename=f'./new_{timeframe}_ema.log', format='%(name)s - %(levelname)s - %(message)s')
logging.warning('{} logging started'.format(datetime.datetime.now().strftime("%x %X")))

def DEMA(data, span, symbol):
  EMA = data[symbol]['close'].ewm(span=span, adjust=False).mean() # data['VOX']['close'] # example of how to get a close column from "data"
  DEMA = 2*EMA - EMA.ewm(span=span, adjust=False).mean()
  return DEMA
#usage example: DEMA(data, 20, 'VOX') gives DEMA of VOX close prices over 20 periods

def get_data_bars(symbols, timeframe, slow, fast):
  data = api.get_barset(symbols, timeframe, limit=50).df 
  for x in symbols:
    data.loc[:, (x, 'slow_ema')] = DEMA(data, slow, x)
    data.loc[:, (x, 'fast_ema')] = DEMA(data, fast, x)
  return data
#usage example: get_data_bars(etflist, '5Min', slow, fast)

def get_signal_bars(symbols, timeframe, slow, fast):
  data = get_data_bars(symbols, timeframe, slow, fast)
  signals = {}
  for x in symbols:
    if data[x].iloc[-1]['fast_ema'] > data[x].iloc[-1]['slow_ema']: # -1 refers to the most recent entry
      signal = 1 # buy when fast crosses above slow
    else: 
      signal = 0 # sell when fast crosses below slow
    signals[x] = signal
  return signals

def order(symbols):
  if account.trading_blocked:
    print('Account is currently restricted from trading.')
    return
  while True:
    clock = api.get_clock()
    if clock.is_open:
      print('Market is open')
      signals = get_signal_bars(symbols, timeframe, slow, fast)
      for signal in signals: # these are the symbols aka the keys not the values
        if signals[signal] == 1: # if signal = buy
          if signal not in [x.symbol for x in api.list_positions()]:
            # https://strftime.org/
            logging.warning(f'{datetime.datetime.now(EST).strftime("%x %X")} {signal} - {signals[signal]}')
            api.submit_order(signal, 1, 'buy', 'market', 'day')
            # print(datetime.datetime.now(EST).strftime("%x %X"), 'buying', signals[signal], signal)
        else: # if signal = sell
          try:
            api.submit_order(signal, 1, 'sell', 'market', 'day')
            logging.warning(f'{datetime.datetime.now(EST).strftime("%x %X")} {signal} - {signals[signal]}')
          except Exception as e:
            print('No sell', signal, e)
      time.sleep(60)
    else:
      print('Market is closed')
      break
    # Get a list of all of our positions.
    portfolio = api.list_positions()
    # Print the quantity of shares for each position.
    for position in portfolio:
      print("{} shares of {}".format(position.qty, position.symbol))

In [12]:
order(symbols)

Market is closed


# Unused functions
There are many Alpaca functionalities not used here. Some are listed below; you can also check out their documentation and website.

In [ ]:
# Unused pt. 1: getting info
# # Check how much money we can use to open new positions.
# print('${} is available as buying power.'.format(account.buying_power))

# # Get a list of all active assets.
# active_assets = api.list_assets(status='active')

# # Filter the assets down to just those on NASDAQ.
# nasdaq_assets = [a for a in active_assets if a.exchange == 'NASDAQ']
# print(nasdaq_assets)

# symbol = 'AAPL'
# symbol_bars = api.get_barset(symbol, 'minute', 1).df.iloc[0]
# symbol_price = symbol_bars[symbol]['close']

# close = barset[etf].df.iloc[0]['close']
# print(f"Last {etf} close: ", close)
# if not api.get_asset(etf).tradable:
#   print(f'We cannot trade {etf}')
# # See how much etf moved in that timeframe.
# week_open = barset[etf][0].o
# week_close = barset[etf][-1].c
# percent_change = (week_close - week_open) / week_open * 100
# print(f'{etf} moved {percent_change}% over the last 5 days')

# # Get the last 100 of our closed orders
# closed_orders = api.list_orders(
#     status='closed',
#     limit=100,
#     nested=True  # show nested multi-leg orders
# )

# # Get only the closed orders for a particular stock
# closed_aapl_orders = [o for o in closed_orders if o.symbol == 'AAPL']
# print(closed_aapl_orders)

# # Get our position in AAPL.
# aapl_position = api.get_position('AAPL')

In [ ]:
# Unused pt. 2: orders
# # Submit a market order to buy 1 share of Apple at market price
# api.submit_order(
#     symbol='AAPL',
#     qty=1,
#     side='buy',
#     type='market',
#     time_in_force='gtc'
# )

# # The security we'll be shorting
# symbol = 'TSLA'

# # Submit a market order to open a short position of one share
# order = api.submit_order(symbol, 1, 'sell', 'market', 'day')
# print("Market order submitted.")

# # Submit a limit order to attempt to grow our short position
# # First, get an up-to-date price for our symbol
# symbol_bars = api.get_barset(symbol, 'minute', 1).df.iloc[0]
# symbol_price = symbol_bars[symbol]['close']
# # Submit an order for one share at that price
# order = api.submit_order(symbol, 1, 'sell', 'limit', 'day', symbol_price)
# print("Limit order submitted.")

# # Wait a second for our orders to fill...
# print('Waiting...')
# time.sleep(1)

# # Check on our position
# position = api.get_position(symbol)
# if int(position.qty) < 0:
#     print(f'Short position open for {symbol}')

# # We could buy a position and add a stop-loss and a take-profit of 5 %
# api.submit_order(
#     symbol=symbol,
#     qty=1,
#     side='buy',
#     type='market',
#     time_in_force='gtc',
#     order_class='bracket',
#     stop_loss={'stop_price': symbol_price * 0.95,
#                'limit_price':  symbol_price * 0.94},
#     take_profit={'limit_price': symbol_price * 1.05}
# )

# # We could buy a position and just add a stop loss of 5 % (OTO Orders)
# api.submit_order(
#     symbol=symbol,
#     qty=1,
#     side='buy',
#     type='market',
#     time_in_force='gtc',
#     order_class='oto',
#     stop_loss={'stop_price': symbol_price * 0.95}
# )

# # We could split it to 2 orders. first buy a stock,
# # and then add the stop/profit prices (OCO Orders)
# api.submit_order(symbol, 1, 'buy', 'limit', 'day', symbol_price)

# # wait for it to buy position and then
# api.submit_order(
#     symbol=symbol,
#     qty=1,
#     side='sell',
#     type='limit',
#     time_in_force='gtc',
#     order_class='oco',
#     stop_loss={'stop_price': symbol_price * 0.95},
#     take_profit={'limit_price': symbol_price * 1.05}
# )

# # Submit a limit order to attempt to sell 1 share of AMD at a
# # particular price ($20.50) when the market opens
# api.submit_order(
#     symbol='AMD',
#     qty=1,
#     side='sell',
#     type='limit',
#     time_in_force='opg',
#     limit_price=20.50
# )

# # Submit a market order and assign it a Client Order ID.
# api.submit_order(
#     symbol='AAPL',
#     qty=1,
#     side='buy',
#     type='market',
#     time_in_force='gtc',
#     client_order_id='my_first_order'
# )

# # Get our order using its Client Order ID.
# my_order = api.get_order_by_client_order_id('my_first_order')
# print('Got order #{}'.format(my_order.id))

# symbol = 'AAPL'
# symbol_bars = api.get_barset(symbol, 'minute', 1).df.iloc[0]
# symbol_price = symbol_bars[symbol]['close']

# # We could buy a position and add a stop-loss and a take-profit of 5 %
# api.submit_order(
#     symbol=symbol,
#     qty=1,
#     side='buy',
#     type='market',
#     time_in_force='gtc',
#     order_class='bracket',
#     stop_loss={'stop_price': symbol_price * 0.95,
#                'limit_price':  symbol_price * 0.94},
#     take_profit={'limit_price': symbol_price * 1.05}
# )

# # We could buy a position and just add a stop loss of 5 % (OTO Orders)
# api.submit_order(
#     symbol=symbol,
#     qty=1,
#     side='buy',
#     type='market',
#     time_in_force='gtc',
#     order_class='oto',
#     stop_loss={'stop_price': symbol_price * 0.95}
# )

# # We could split it to 2 orders. first buy a stock,
# # and then add the stop/profit prices (OCO Orders)
# api.submit_order(symbol, 1, 'buy', 'limit', 'day', symbol_price)

# # wait for it to buy position and then
# api.submit_order(
#     symbol=symbol,
#     qty=1,
#     side='sell',
#     type='limit',
#     time_in_force='gtc',
#     order_class='oco',
#     stop_loss={'stop_price': symbol_price * 0.95},
#     take_profit={'limit_price': symbol_price * 1.05}
# )